In [1]:
# ALWAYS RUN THIS CELL
# widen jupyter notebook window
from IPython.display import display, HTML
display(HTML("<style>.container {width:95% !important; }</style>"))

This requires having `bnpm`. Get it with `pip install bnpm` or here: https://github.com/RichieHakim/basic_neural_processing_modules

In [3]:
from bnpm import server

In [6]:
import os
from pathlib import Path

import paramiko
# from scp import SCPClient

import getpass
import gc
import time

------------
# ==== Start ====

## Prepare username + password

In [8]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')

use_localSshKey = True

pw = server.pw_encode(getpass.getpass(prompt='Password: ')) if use_localSshKey==False else None

path_sshKey = '/home/rich/.ssh/id_rsa' if use_localSshKey else None

Username: rh183


## Prepare `sftp` and `ssh` objects

In [43]:
## initialize ssh_transfer
ssh_t = server.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)
ssh_t.o2_connect(
    hostname=remote_host_transfer,
    username=username,
    password=server.pw_decode(pw),
    key_filename=path_sshKey,
    look_for_keys=False,
    passcode_method=1,
    verbose=0,
    skip_passcode=False,    
)

In [10]:
## initialize ssh_compute
ssh_c = server.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)
ssh_c.o2_connect(
    hostname=remote_host_compute,
    username=username,
    password=server.pw_decode(pw),
    key_filename=path_sshKey,
    look_for_keys=False,
    passcode_method=1,
    verbose=0,
    skip_passcode=False,    
)

In [44]:
sftp = server.sftp_interface(ssh_client=ssh_t.client)

In [78]:
prompt_snip = f'[{username}'

In [81]:
import functools

send_expect = functools.partial(ssh_c.send_expect, str_success=prompt_snip)

## Start a small interactive node

In [79]:
send_expect(f'srun --pty -p interactive -t 0-00:30:00 --mem=4G /bin/bash');

srun --pty -p interactive -t 0-00:30:00 --mem=4G /bin/bash

srun: job 11964478 queued and waiting for resources

srun: job 11964478 has been allocated resources

(base) [rh183@compute-a-16-164 face-rhythm]$ 


## Pull/update repo

In [82]:
send_expect('cd /n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm')
send_expect('git pull');

ich/github_repos/face-rhythmatini/r 

(base) [rh183@compute-a-16-164 face-rhythm]$ 
g
it pull

Already up-to-date.
(base) [rh183@compute-a-16-164 face-rhythm]$ 


('Already up-to-date.\r\n(base) [rh183@compute-a-16-164 face-rhythm]$ ', True)

## Activate environment

In [83]:
name_env = '/n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'

send_expect(f'conda activate {name_env}');

o/sabatini/rich/virtual_envs/FRrobi 

ce-rhythm]$ s/neurobio/sabatini/rich/virtual_envs/FR) [rh183@compute-a-16-164 fac


## Prepare directories

In [84]:
import numpy as np

mouse = 'mouse_0322N'
date = '20230430'

name_slurm = mouse + '_' + date[-4:]

dir_videos      = str(Path('/n/data1/hms/neurobio/sabatini/gyu/data/full_data') / mouse / date / 'camera_data' / 'exp' / 'cam4')

dir_ROIs        = str(Path('/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm') / mouse / 'run_20230701' / 'ROIs_prealigned' / f'{mouse}__{date}' / 'ROIs.h5')

dir_saveOutputs = str(Path('/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm') / mouse / 'run_20230701' / 'FR')


name_job = 'jobNum_'


path_script_remote      = str(Path(r'/n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm/scripts/pipeline_basic.py'))  ## path to a copy of the script (.py) file to run within the repo on the server

path_dispatcher_local   = str(Path(r'/media/rich/Home_Linux_partition/github_repos/face-rhythm/batch_run/').resolve() / 'dispatcher.py')  ## path to the dispatcher.py file on local computer
path_dispatcher_remote  = (Path(dir_saveOutputs) / 'dispatcher.py').as_posix()  ## path to where you want the dispatcher.py file to be copied onto the server

## Prepare commands

In [85]:
## Expectation for dispatch args: 
# path_script, dir_save, dir_videos, dir_ROIs, name_job, name_slurm, name_env


commands = {
#     'make_dir': f"mkdir -p {dir_data_remote}",
#     'copy': f"cp -r {dir_s2p_MICROSCOPE} {dir_data_remote}",
#     'dispatch': f"python {str(path_dispatcher_remote)} {dir_saveOutputs} {path_script_remote} {name_job} {name_slurm} {dir_videos}",  ## dispatcher expecting these args as inputs
    'dispatch': f"\
python3 \
{path_dispatcher_remote} \
{path_script_remote} \
{dir_saveOutputs} \
{dir_videos} \
{dir_ROIs} \
{name_job} \
{name_slurm} \
{name_env}",
}
display(commands)

{'dispatch': 'python3 /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/FR/dispatcher.py /n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm/scripts/pipeline_basic.py /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/FR /n/data1/hms/neurobio/sabatini/gyu/data/full_data/mouse_0322N/20230430/camera_data/exp/cam4 /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/ROIs_prealigned/mouse_0322N__20230430/ROIs.h5 jobNum_ mouse_0322N_0430 /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'}

## Upload `dispatcher.py` file

In [86]:
sftp.mkdir_p(str(Path(path_dispatcher_remote).parent))
sftp.sftp.put(str(path_dispatcher_local), str(path_dispatcher_remote));

## Dispatch

In [91]:
send_expect(commands['dispatch']);

e_0322N/run_20230701/FR/dispatcher.ini/rich/analysis/faceRhythm/mous 
ini/rich/virtual_envs/FR.h5 jobNum_ mouse_0322N_0430 /n/data1/hms/neurobio/sabat 

dispatcher environment: /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

Copying /n/data1/hms/neurobio/sabatini/rich/github_repos/face-rhythm/scripts/pipeline_basic.py to /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/FR/pipeline_basic.py
Submitting job: jobNum_ 0

sbatch: Setting QOS to gpuquad_qos
Submitted batch job 11965136

ce-rhythm]$ s/neurobio/sabatini/rich/virtual_envs/FR) [rh183@compute-a-16-164 fac


### check on job

In [90]:
ssh_c.send_expect(f'squeue -u {username} --Format=jobid,name,partition,state,timeused,starttime,timelimit,reasonlist,minmemory,mincpus,gres');

me,timelimit,reasonlist,minmemory,mincpus,gresstate,timeused,startti 


JOBID               NAME                PARTITION           STATE               TIME                START_TIME          TIME_LIMIT          NODELIST(REASON)    MIN_MEMORY          MIN_CPUS            TRES_PER_NODE       
11964784            mouse_0322N_0430    gpu_requeue         RUNNING             0:02                2023-07-02T09:57:56 30:00               compute-gc-17-245   10G                 4                   gres:gpu:rtx6000:1  
11964478            bash                interactive         RUNNING             3:42                2023-07-02T09:54:16 30:00               compute-a-16-164    4G                  1                   N/A                 
ce-rhythm]$ s/neurobio/sabatini/rich/virtual_envs/FR) [rh183@compute-a-16-164 fac


In [89]:
prompt_snip

'[rh183'

## Clean up

In [ ]:
## delete password
del pw
gc.collect()

## close sftp
if sftp: sftp.close()
if transport: transport.close()
del sftp
del transport
    
## close ssh
ssh_t.close()
del ssh_t
ssh_c.close()
del ssh_c
gc.collect()